In [1]:
#Collect all the games with stats from 2008
import pandas as pd
import numpy as np
pt = pd.read_csv("../data/B1_FavGames.csv")
KBBP = pd.read_csv("../data/B3_KPBPIBRP.csv").fillna(0)
pt = pt.loc[(pt['Year']>=2008) & (pt['Round'] !='PI')]
pt['Round'] = pt['Round'].astype(int)
pt = pt.iloc[:,0:10]
pt['PFSeed'] = pt['AFSeed']
pt['PFTeam'] = pt['AFTeam']
pt['PFScore'] = pt['AFScore']
pt['PUSeed'] = pt['AUSeed']
pt['PUTeam'] = pt['AUTeam']
pt['PUScore'] = pt['AUScore']
pts = pt.merge(KBBP, left_on=['Year','PFTeam'],right_on=['Year','Team'],how='left')
pts = pts.merge(KBBP, left_on=['Year','PUTeam'],right_on=['Year','Team'],how='left')

pts.to_csv('../data/g1_merged.csv')



In [7]:
# Pick a random Year
from random import choice
p_year = choice([i for i in range(2008,2021) if i not in [2020]])
p_year

2008

In [8]:
# Back Predict Round 1
# The Model from all years
expl = pts.select_dtypes(exclude=['object']).drop(['AFSeed', 'AUSeed','AFScore','AUScore','PFScore','PUScore'],axis=1)
respF = pts['PFScore']
respU = pts['PUScore']
from sklearn.linear_model import ElasticNet
ENMF = ElasticNet(alpha=.284,l1_ratio=.5)
ENMF.fit(expl,respF)
ENMU = ElasticNet(alpha=.284,l1_ratio=.5)
ENMU.fit(expl,respU)

# Predict p_year
exply = expl[expl['Year']==p_year]
PF = ENMF.predict(exply)
PU = ENMU.predict(exply)

expl.columns


Index(['Year', 'Round', 'Game', 'PFSeed', 'PUSeed', 'AdjEM_x', 'AdjO_x',
       'AdjD_x', 'AdjT_x', 'SOSEM_x', 'SOSO_x', 'SOSD_x', 'BPIO_x', 'BPID_x',
       'W_x', 'L_x', 'Pts_x', 'Opp_x', 'MOV_x', 'SOS_x', 'OSRS_x', 'DSRS_x',
       'SRS_x', 'PASE_x', 'AdjEM_y', 'AdjO_y', 'AdjD_y', 'AdjT_y', 'SOSEM_y',
       'SOSO_y', 'SOSD_y', 'BPIO_y', 'BPID_y', 'W_y', 'L_y', 'Pts_y', 'Opp_y',
       'MOV_y', 'SOS_y', 'OSRS_y', 'DSRS_y', 'SRS_y', 'PASE_y'],
      dtype='object')

In [23]:
# Evaluate the Prediction
pty = pt[pt['Year']==p_year]
pty.index = pty.Game
for x in (1,33):
    pty.loc[x, pty['PFScore']] = np.around(PF[x-1],2)

C:\Users\harmk\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
pty.loc[pty['Round'], 'PUScore'] = np.around(PU,2)

In [10]:
for x in range(1,33):
    pty.loc[x,'AWSeed'] = np.where(pty.loc[x,'AFScore']>=pty.loc[x,'AUScore'],pty.loc[x,'AFSeed'],pty.loc[x,'AUSeed'])
    pty.loc[x,'AWTeam'] = np.where(pty.loc[x,'AFScore']>=pty.loc[x,'AUScore'],pty.loc[x,'AFTeam'],pty.loc[x,'AUTeam'])
    pty.loc[x,'PWSeed'] = np.where(pty.loc[x,'PFScore']>=pty.loc[x,'PUScore'],pty.loc[x,'PFSeed'],pty.loc[x,'PUSeed'])
    pty.loc[x,'PWTeam'] = np.where(pty.loc[x,'PFScore']>=pty.loc[x,'PUScore'],pty.loc[x,'PFTeam'],pty.loc[x,'PUTeam'])
    pty.loc[x,'ESPN'] = np.where(pty.loc[x,'AWTeam']==pty.loc[x,'PWTeam'],10,0)
print(p_year)
pty[pty['Round']==1]['ESPN'].sum()


2008


C:\Users\harmk\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\harmk\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


250.0

In [12]:
pty.to_csv('../data/tal.csv')